In [ ]:
import torch
from titan import NeuralMemory

# create dummy keys, queries and values
keys = torch.randn(2, 5, 4)
queries = keys
values = torch.randn(2, 5, 4)


# create a NeuralMemory instance
memory = NeuralMemory(input_shape=keys.shape, hidden_units=3)

output=None
for i in range(5):
    output = memory.forward_inference(keys, queries, values)
    diff = values-output
    print(diff.mean().item())

print("-----------------------")
for i in range(5):
    output = memory.forward_inference(keys, queries, values)
    memory.apply_cached_updates()
    diff = values-output
    print(diff.mean().item())

output = memory.forward_inference(keys, queries, values)

print("input shapes: ", keys.shape, queries.shape, values.shape)
print("output shape: ", output.shape)

Building module with input shape  torch.Size([2, 5, 4])
shape after 1st layer:  torch.Size([2, 5, 3])
shape after final layer:  torch.Size([2, 5, 4])
Building module with input shape  torch.Size([2, 5, 5])
shape after 1st layer:  torch.Size([2, 5, 5])
shape after final layer:  torch.Size([2, 5, 5])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x4 and 5x5)

In [42]:
import torch
import torch.nn.functional as F
from titan import TryBackprop, NeuralMemory

# Create a random input tensor with shape (batch_size, seq_len, feature_dim)
x = torch.randn(2, 5, 4)
model = TryBackprop(input_shape=x.shape, hidden_units=5)

# Register a hook to zero out outer-loss gradients for Neural Memory parameters.
for param in model.layer_dict["Neural_Memory"].parameters():
    param.register_hook(lambda grad: torch.zeros_like(grad))

output = model(x)
target = torch.randn_like(output)
loss = F.mse_loss(output, target)
loss.backward()
model.layer_dict["Neural_Memory"].apply_cached_updates()
model.layer_dict["Neural_Memory"].reset_computational_history()

# Print out the loss and one example of a parameter's gradient
print("Loss:", loss.item())
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"{name} grad mean: {param.grad.mean().item():.6f}")
    else:
        print(f"{name} has no gradient")


x2 = torch.randn(2, 5, 4)
target2 = torch.randn_like(output)
output2 = model(x2)
loss = F.mse_loss(output2, target2)
loss.backward()
model.layer_dict["Neural_Memory"].apply_cached_updates()
model.layer_dict["Neural_Memory"].reset_computational_history()
print("------------------------")
# Print out the loss and one example of a parameter's gradient
print("Loss:", loss.item())
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"{name} grad mean: {param.grad.mean().item():.6f}")
    else:
        print(f"{name} has no gradient")



Building module with input shape  torch.Size([2, 5, 5])
shape after 1st layer:  torch.Size([2, 5, 5])
shape after final layer:  torch.Size([2, 5, 5])
Loss: 1.167250394821167
layer_dict.Queries.weight grad mean: 0.000503
layer_dict.Queries.bias grad mean: 0.034973
layer_dict.Keys.weight grad mean: 0.000000
layer_dict.Keys.bias grad mean: 0.003391
layer_dict.Values.weight grad mean: 0.000000
layer_dict.Values.bias grad mean: 0.000671
layer_dict.Neural_Memory.layer_dict.Input_Layer.weight grad mean: 0.000000
layer_dict.Neural_Memory.layer_dict.Input_Layer.bias grad mean: 0.000000
layer_dict.Neural_Memory.layer_dict.bn0.weight grad mean: 0.000000
layer_dict.Neural_Memory.layer_dict.bn0.bias grad mean: 0.000000
layer_dict.Neural_Memory.layer_dict.Output_Layer.weight grad mean: 0.000000
layer_dict.Neural_Memory.layer_dict.Output_Layer.bias grad mean: 0.000000
layer_dict.Output_Layer.weight grad mean: -0.014919
layer_dict.Output_Layer.bias grad mean: 0.067435
------------------------
Loss: 1.

In [ ]:
import torch
from titan import NeuralMemory, TryBackprop

# create dummy keys, queries and values
keys = torch.randn(2, 5, 4)
queries = keys
values = torch.randn(2, 5, 4)


# create a NeuralMemory instance
model = TryBackprop(input_shape=keys.shape, hidden_units=3)

output=None
for i in range(5):
    output = model.layer_dict["Neural_Memory"].forward_inference(keys, queries, values)
    model.layer_dict["Neural_Memory"].apply_cached_updates()
    diff = values-output
    print(diff.mean().item())

print("-----------------------")
for i in range(5):
    output = model.layer_dict["Neural_Memory"].forward_inference(keys, queries, values)
    model.layer_dict["Neural_Memory"].apply_cached_updates()
    diff = values-output
    print(diff.mean().item())

output = memory.forward_inference(keys, queries, values)

print("input shapes: ", keys.shape, queries.shape, values.shape)
print("output shape: ", output.shape)

In [1]:
import torch

x = torch.randn(32, 100, 17) 
y = torch.randn(32, 100, 17)

b = torch.bmm(x, y.transpose(2, 1))

print(b.shape)

torch.Size([32, 100, 100])
